In [1]:
#https://drive.google.com/file/d/1EVwP6MUQAeSnGM-ywBRHspCn_OWEfzJN/view?usp=sharing
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1EVwP6MUQAeSnGM-ywBRHspCn_OWEfzJN" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1EVwP6MUQAeSnGM-ywBRHspCn_OWEfzJN" -o train.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    333      0 --:--:--  0:00:01 --:--:--   333
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  955k  100  955k    0     0   592k      0  0:00:01  0:00:01 --:--:-- 10.1M


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('train.csv')
df.head()

,id,time,x,y
0,1,0,800,0
1,1,780,780,0
2,1,1572,792,0
3,1,2392,820,0
4,1,3196,804,0


In [4]:
df['marker'] = np.multiply(df[['id','y']].groupby('id').agg(np.cumsum).values.ravel(), df['y'].values)
df['start'] = ((df['y'].shift(1, fill_value=0) == 0).values & (df['y'] == 1)).values
df['end'] = ((df['y'].shift(-1, fill_value=0) == 0).values & (df['y'] == 1)).values

q = df.loc[df['end'],'marker'] - df.loc[df['end'],'start']
print('аномалий: ', q.shape[0], 'средняя длина: ', int(q.mean()*100.0)*0.01, 'min: ', q.min(), 'max: ', q.max())

аномалий:  791 средняя длина:  25.060000000000002 min:  6 max:  81


In [5]:
import plotly.express as px
import plotly.graph_objects as go

for q in [163]:
    t = df.loc[df['id'] == q].sort_values('time').reset_index(drop=True)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=t['time'], y=t['x'] - t['x'].mean(),
                        mode='lines',
                        name='lines'))
    qt = t.loc[t.y==1].reset_index(drop=True)
    fig.add_trace(go.Scatter(x=qt['time'], y=qt['x'] - t['x'].mean(),
                        mode='markers', name='markers'))
    print(t.x.mean())
    fig.show()


728.0415584415584


In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import torch
from torch.utils.data import Dataset

def scale_ts(vls):
    sc = StandardScaler()
    return sc.fit_transform(vls.reshape(-1,1)).ravel()
def scale_time_ts(vls):
    sc = MinMaxScaler()
    return sc.fit_transform(vls.reshape(-1,1)).ravel()

class CardioDataset(Dataset):
    def __init__(self, df, win_size=32):
        self.df = df.sort_values(['id','time']).reset_index(drop=True).copy()
        self.df['time'] = df.groupby('id')['time'].agg('diff').fillna(0).values
        self.df['time'] = scale_time_ts(self.df['time'].values)
        self.win_size = win_size

        self.point_indexes = []
        self.win_lens = []
        dfs = []
        total_len = 0
        for q,qdf in self.df.groupby('id'):
            #qdf['x'] = scale_ts(qdf['x'].values)
            for i in range(max(1, qdf.shape[0] - win_size + 1)):
                self.point_indexes.append(i + total_len)
                if i + win_size > qdf.shape[0]:
                    self.win_lens.append(qdf.shape[0] - i)
                else:
                    self.win_lens.append(win_size)
            total_len += qdf.shape[0]
            dfs.append(qdf)
        self.df = pd.concat(dfs, ignore_index=True).reset_index(drop=True)

    def __len__(self):
        return len(self.point_indexes)

    def __getitem__(self, idx):
        i0 = self.point_indexes[idx]
        i1 = i0 + self.win_lens[idx]

        x_mat = np.zeros((self.win_size,2))
        y_mat = np.zeros(self.win_size)
        x_mat[-self.win_lens[idx]:,0] = scale_ts(self.df.iloc[i0:i1].x.values)
        x_mat[-self.win_lens[idx]:,1] = self.df.iloc[i0:i1].time.values
        y_mat[-self.win_lens[idx]:] = self.df.iloc[i0:i1].y.values
        
        return {"x": x_mat,
                "y": y_mat,
                "start": i0,
                "end": i1
               }

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

class CardioRnn(nn.Module):
    def __init__(self, win_size, output_size, rnn_units = 32):
        super().__init__()
        self._gru = nn.GRU(input_size=2, 
                           num_layers=4,
                           dropout=0.3,
                           hidden_size=rnn_units, 
                           batch_first=True, 
                           bidirectional=True)

        self._head = nn.Linear(in_features = 4 * rnn_units, 
                               out_features=output_size)
        
        self.fcdrop2 = nn.Dropout(p=0.3)
        
    def forward(self, x_feats):
        encoded, _ = self._gru(x_feats)
        x2 = torch.max(encoded, axis=1).values
        x2 = torch.squeeze(x2)

        x3 = torch.mean(encoded, axis=1)
        x3 = torch.squeeze(x3)

        x = torch.cat([x2,x3],-1)
        x = F.relu(x)
        x = self.fcdrop2(x)
        x = self._head(x)
        return x

In [8]:
from torch.utils.data import DataLoader
from torch.utils.data import RandomSampler, SequentialSampler

model = CardioRnn(32, 32).to(device)

ds = CardioDataset(df, 32)
print(len(ds))
train_sampler = SequentialSampler(ds)
batch_size = 1
train_dl = DataLoader(ds, sampler=train_sampler, batch_size=batch_size, num_workers=2)
for x in train_dl:
    print(model(x['x'].float().to(device)))
    break

53388
tensor([ 0.0985, -0.0467, -0.1338,  0.0092,  0.1249,  0.0026,  0.0857,  0.2297,
         0.0979, -0.0049,  0.0813,  0.0746, -0.0838,  0.1438,  0.0042,  0.0101,
        -0.1735,  0.0255, -0.0758, -0.1046,  0.1656,  0.0018, -0.0560,  0.0851,
         0.0245,  0.0890,  0.0751,  0.0363,  0.1544, -0.1678,  0.1092, -0.0587],
       device='cuda:0', grad_fn=<AddBackward0>)


In [9]:
from sklearn.metrics import roc_curve, precision_recall_curve, f1_score

def threshold_search(y_true, y_proba):
    precision , recall, thresholds = precision_recall_curve(y_true, y_proba)
    thresholds = np.append(thresholds, 1.001) 
    F = 2 / (1/precision + 1/recall)
    best_score = np.max(F)
    best_th = thresholds[np.argmax(F)]
    return best_th , best_score

In [10]:
%%writefile lookahead.py
# Lookahead implementation from https://github.com/rwightman/pytorch-image-models/blob/master/timm/optim/lookahead.py

""" Lookahead Optimizer Wrapper.
Implementation modified from: https://github.com/alphadl/lookahead.pytorch
Paper: `Lookahead Optimizer: k steps forward, 1 step back` - https://arxiv.org/abs/1907.08610
"""
import torch
from torch.optim.optimizer import Optimizer
from collections import defaultdict

class Lookahead(Optimizer):
    def __init__(self, base_optimizer, alpha=0.5, k=6):
        if not 0.0 <= alpha <= 1.0:
            raise ValueError(f'Invalid slow update rate: {alpha}')
        if not 1 <= k:
            raise ValueError(f'Invalid lookahead steps: {k}')
        defaults = dict(lookahead_alpha=alpha, lookahead_k=k, lookahead_step=0)
        self.base_optimizer = base_optimizer
        self.param_groups = self.base_optimizer.param_groups
        self.defaults = base_optimizer.defaults
        self.defaults.update(defaults)
        self.state = defaultdict(dict)
        # manually add our defaults to the param groups
        for name, default in defaults.items():
            for group in self.param_groups:
                group.setdefault(name, default)

    def update_slow(self, group):
        for fast_p in group["params"]:
            if fast_p.grad is None:
                continue
            param_state = self.state[fast_p]
            if 'slow_buffer' not in param_state:
                param_state['slow_buffer'] = torch.empty_like(fast_p.data)
                param_state['slow_buffer'].copy_(fast_p.data)
            slow = param_state['slow_buffer']
            slow.add_(group['lookahead_alpha'], fast_p.data - slow)
            fast_p.data.copy_(slow)

    def sync_lookahead(self):
        for group in self.param_groups:
            self.update_slow(group)

    def step(self, closure=None):
        # print(self.k)
        #assert id(self.param_groups) == id(self.base_optimizer.param_groups)
        loss = self.base_optimizer.step(closure)
        for group in self.param_groups:
            group['lookahead_step'] += 1
            if group['lookahead_step'] % group['lookahead_k'] == 0:
                self.update_slow(group)
        return loss

    def state_dict(self):
        fast_state_dict = self.base_optimizer.state_dict()
        slow_state = {
            (id(k) if isinstance(k, torch.Tensor) else k): v
            for k, v in self.state.items()
        }
        fast_state = fast_state_dict['state']
        param_groups = fast_state_dict['param_groups']
        return {
            'state': fast_state,
            'slow_state': slow_state,
            'param_groups': param_groups,
        }

    def load_state_dict(self, state_dict):
        fast_state_dict = {
            'state': state_dict['state'],
            'param_groups': state_dict['param_groups'],
        }
        self.base_optimizer.load_state_dict(fast_state_dict)

        # We want to restore the slow state, but share param_groups reference
        # with base_optimizer. This is a bit redundant but least code
        slow_state_new = False
        if 'slow_state' not in state_dict:
            print('Loading state_dict from optimizer without Lookahead applied.')
            state_dict['slow_state'] = defaultdict(dict)
            slow_state_new = True
        slow_state_dict = {
            'state': state_dict['slow_state'],
            'param_groups': state_dict['param_groups'],  # this is pointless but saves code
        }
        super(Lookahead, self).load_state_dict(slow_state_dict)
        self.param_groups = self.base_optimizer.param_groups  # make both ref same container
        if slow_state_new:
            # reapply defaults to catch missing lookahead specific ones
            for name, default in self.defaults.items():
                for group in self.param_groups:
                    group.setdefault(name, default)

Writing lookahead.py


In [11]:
%%writefile ralamb.py
import torch, math
from torch.optim.optimizer import Optimizer

# RAdam + LARS
class Ralamb(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(Ralamb, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(Ralamb, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('Ralamb does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                # Decay the first and second moment running average coefficient
                # m_t
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                # v_t
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]

                if state['step'] == buffered[0]:
                    N_sma, radam_step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        radam_step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        radam_step_size = 1.0 / (1 - beta1 ** state['step'])
                    buffered[2] = radam_step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                radam_step = p_data_fp32.clone()
                if N_sma >= 5:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    radam_step.addcdiv_(-radam_step_size * group['lr'], exp_avg, denom)
                else:
                    radam_step.add_(-radam_step_size * group['lr'], exp_avg)

                radam_norm = radam_step.pow(2).sum().sqrt()
                weight_norm = p.data.pow(2).sum().sqrt().clamp(0, 10)
                if weight_norm == 0 or radam_norm == 0:
                    trust_ratio = 1
                else:
                    trust_ratio = weight_norm / radam_norm

                state['weight_norm'] = weight_norm
                state['adam_norm'] = radam_norm
                state['trust_ratio'] = trust_ratio

                if N_sma >= 5:
                    p_data_fp32.addcdiv_(-radam_step_size * group['lr'] * trust_ratio, exp_avg, denom)
                else:
                    p_data_fp32.add_(-radam_step_size * group['lr'] * trust_ratio, exp_avg)

                p.data.copy_(p_data_fp32)

        return loss

Writing ralamb.py


In [12]:
from lookahead import *
from ralamb import * 

def Over9000(params, alpha=0.5, k=6, *args, **kwargs):
     ralamb = Ralamb(params, *args, **kwargs)
     return Lookahead(ralamb, alpha, k)

RangerLars = Over9000

In [13]:
class CardioCnn(nn.Module):
    def __init__(self, win_size, output_size, 
                 out_chan = 64, 
                 top_classifier_units = 64,
                 rnn_units = 8,
                 kern_sizes = [3,5,7,9]):
        super().__init__()
        self.cnn0 = nn.Conv1d(2, out_chan, kern_sizes[0])
        self.cnn1 = nn.Conv1d(2, out_chan, kern_sizes[1])
        self.cnn2 = nn.Conv1d(2, out_chan, kern_sizes[2])
        self.cnn3 = nn.Conv1d(2, out_chan, kern_sizes[3])

        self._gru = nn.GRU(input_size=2, 
                           num_layers=4,
                           dropout=0.3,
                           hidden_size=rnn_units, 
                           batch_first=True, 
                           bidirectional=True)

        self.sum_chans = (win_size*4 - np.sum(kern_sizes) + 4) + 2*rnn_units

        self._head = nn.Sequential(
            nn.LayerNorm(self.sum_chans),
            nn.Linear(self.sum_chans, top_classifier_units),
            nn.LayerNorm(top_classifier_units),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(top_classifier_units, top_classifier_units),
            nn.LayerNorm(top_classifier_units),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(top_classifier_units, output_size),            
        ) 
        
    def forward(self, x_feats):
        encoded,_ = self._gru(x_feats)
        encoded0 = self.cnn0(x_feats.swapaxes(1,2))
        encoded1 = self.cnn1(x_feats.swapaxes(1,2))
        encoded2 = self.cnn2(x_feats.swapaxes(1,2))
        encoded3 = self.cnn3(x_feats.swapaxes(1,2))

        x0 = torch.max(encoded0, axis=1).values
        x0 = torch.squeeze(x0)
        x1 = torch.mean(encoded0, axis=1)
        x1 = torch.squeeze(x1)

        x2 = torch.max(encoded1, axis=1).values
        x2 = torch.squeeze(x2)
        x3 = torch.mean(encoded1, axis=1)
        x3 = torch.squeeze(x3)

        x4 = torch.max(encoded2, axis=1).values
        x4 = torch.squeeze(x4)
        x5 = torch.mean(encoded2, axis=1)
        x5 = torch.squeeze(x5)

        x6 = torch.max(encoded3, axis=1).values
        x6 = torch.squeeze(x6)
        x7 = torch.mean(encoded3, axis=1)
        x7 = torch.squeeze(x7)

        x8 = torch.max(encoded, axis=1).values
        x8 = torch.squeeze(x8)

        x = torch.cat([x0,x2,x4,x6,x8],-1)
        x = self._head(x)
        return x

In [23]:
import joblib 
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
import tqdm

WIN_SIZE = 25

kf = KFold(n_splits=5, shuffle=True, random_state=42)
ifold = 0
fold_metrics = []
for tr_id, va_id in kf.split(df['id'].unique()):
    train_df = df.loc[df['id'].isin(tr_id)].reset_index(drop=True)
    valid_df = df.loc[df['id'].isin(va_id)].reset_index(drop=True)

    train_ds = CardioDataset(train_df, WIN_SIZE)
    valid_ds = CardioDataset(valid_df, WIN_SIZE)

    train_sampler = RandomSampler(train_ds)
    valid_sampler = SequentialSampler(valid_ds)
    batch_size = 256
    train_dl = DataLoader(train_ds, sampler=train_sampler, batch_size=batch_size, num_workers=0)
    valid_dl = DataLoader(valid_ds, sampler=valid_sampler, batch_size=batch_size, num_workers=0)

    model = CardioCnn(WIN_SIZE, WIN_SIZE).to(device)

    nepochs = 2

    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters(),
                          lr = 1e-3                          
                       )
    optimizer = Lookahead(optimizer, 0.5, 6)
    #optimizer = RangerLars(model.parameters())

    for epoch in range(nepochs):
        model.train();
        optimizer.zero_grad()
        i = 0
        train_losses = []
        for x in train_dl:
            out = model(x['x'].float().to(device))
            loss = criterion(out, x['y'].float().to(device)) 
            loss.backward()
            optimizer.step() 
            optimizer.zero_grad()
            i += 1
        #torch.save(model.state_dict(), 'model_'+str(epoch) + '.pth')
            train_losses.append(loss.item())
        
        valid_losses = []
        valid_df_predict = np.zeros((valid_df.shape[0],WIN_SIZE))
        model.eval();
        for xx in valid_dl:
            out = model(xx['x'].float().to(device))
            valid_loss = criterion(out, xx['y'].float().to(device)) 
            valid_losses.append(valid_loss.item())
            out = torch.nn.Softmax(dim=1)(out)
            for j in range(xx['x'].shape[0]):
                i0,i1 = xx["start"][j],xx["end"][j]
                s = i1 - i0
                valid_df_predict[i0:i1, i0 % WIN_SIZE] = out[j].detach().cpu().numpy()[-s:]
        y_proba = np.amax(valid_df_predict, 1)
        y_true = valid_df.y.values

        #best_th , best_score = threshold_search(y_true, y_proba)
        best_th = 0.066
        best_score = f1_score(y_true.astype(int), (y_proba > best_th).astype(int)) #, average='micro'
        print('fold', ifold, 'epoch', epoch, 'best_th', best_th, 'f1-score', best_score) #'step', i,
        print('fold', ifold, 'epoch', epoch, 'train_loss', np.mean(train_losses), 'valid_loss', np.mean(valid_losses)) 
        model.train();

    fold_metrics.append(best_score)
    ifold += 1

fold 0 epoch 0 best_th 0.066 f1-score 0.1290704558910598
fold 0 epoch 0 train_loss 0.44643505993268345 valid_loss 0.38455644838119807
fold 0 epoch 1 best_th 0.066 f1-score 0.5797485109199206
fold 0 epoch 1 train_loss 0.34427988756176653 valid_loss 0.29045598806911394
fold 1 epoch 0 best_th 0.066 f1-score 0.003997335109926716
fold 1 epoch 0 train_loss 0.4572673811428789 valid_loss 0.3307643223042582
fold 1 epoch 1 best_th 0.066 f1-score 0.5172087148721187
fold 1 epoch 1 train_loss 0.38047993247923645 valid_loss 0.2679646294198784
fold 2 epoch 0 best_th 0.066 f1-score 0.023994354269583625
fold 2 epoch 0 train_loss 0.4489609327995115 valid_loss 0.3395664181974199
fold 2 epoch 1 best_th 0.066 f1-score 0.4728761514841351
fold 2 epoch 1 train_loss 0.3889163606282737 valid_loss 0.2701900555027856
fold 3 epoch 0 best_th 0.066 f1-score 0.0
fold 3 epoch 0 train_loss 0.410651409486581 valid_loss 0.679981196920077
fold 3 epoch 1 best_th 0.066 f1-score 0.6426553672316384
fold 3 epoch 1 train_loss 0

In [24]:
print(fold_metrics, np.mean(fold_metrics))

[0.5797485109199206, 0.5172087148721187, 0.4728761514841351, 0.6426553672316384, 0.6319165998396151] 0.5688810688694856
